In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sn
import numpy as np
import seaborn as sns
import plotly.express as px

In [74]:
data = pd.read_csv("data/laptop_final.csv")
print(data.head(5))
print(data.columns)

                                                link  \
0  https://laptopmedia.com/laptop-specs/asus-rog-...   
1  https://laptopmedia.com/laptop-specs/asus-rog-...   
2  https://laptopmedia.com/laptop-specs/asus-rog-...   
3  https://laptopmedia.com/laptop-specs/asus-rog-...   
4  https://laptopmedia.com/laptop-specs/asus-rog-...   

                       name  Display Size  Resolution: width  \
0        ASUS ROG Strix G17          17.3             2560.0   
1  ASUS ROG Zephyrus Duo 16          16.0             2560.0   
2  ASUS ROG Zephyrus Duo 16          16.0             2560.0   
3  ASUS ROG Zephyrus Duo 16          16.0             2560.0   
4        ASUS ROG Strix G17          17.3             2560.0   

   Resolution: height  Refresh Rate       Panel Type  Disk Capacity  SSD  HDD  \
0              1440.0         240.0     IPS + G-Sync         2000.0  1.0  0.0   
1              1600.0         240.0   IPS + FreeSync        16000.0  1.0  0.0   
2              1600.0         240.0

In [75]:
print(data["name"])

0               ASUS ROG Strix G17
1         ASUS ROG Zephyrus Duo 16
2         ASUS ROG Zephyrus Duo 16
3         ASUS ROG Zephyrus Duo 16
4               ASUS ROG Strix G17
                    ...           
289166               Acer Aspire 5
289167                       HP 17
289168           HP 17 (17-by4000)
289169                       HP 17
289170           HP 17 (17-by4000)
Name: name, Length: 289171, dtype: object


In [76]:
data = data.dropna()

# Hypothesese & EDA

# 1,Mean cost to increase point in laptop

In [79]:
# Find the price segment
print(data['Cost'].value_counts())
# Divide cost into quantile-based ranges
data['Price Segment'] = pd.qcut(data['Cost'], q=5, labels=['Low', 'Lower Mid', 'Mid', 'Upper Mid', 'High'])
# Check distribution
print(data['Price Segment'].value_counts())

Cost
1499.00    290
999.00     268
1299.00    263
1399.00    252
1599.00    242
          ... 
738.97       1
708.97       1
758.94       1
1592.99      1
364.57       1
Name: count, Length: 4149, dtype: int64
Price Segment
Mid          6390
Lower Mid    6332
Low          6329
Upper Mid    6298
High         6293
Name: count, dtype: int64


In [80]:
data['Brand'] = data['name'].apply(lambda x : x.split()[0])

In [81]:
grouped = data.groupby(['Price Segment', 'Brand']).agg({
    'Work Score': 'mean',
    'Play Score': 'mean',
    'Display Score': 'mean',
    'Portability Score': 'mean',
    'Total Score': 'mean'
}).reset_index()

best_work = grouped.loc[grouped.groupby('Price Segment')['Work Score'].idxmax()]
best_play = grouped.loc[grouped.groupby('Price Segment')['Play Score'].idxmax()]
best_display = grouped.loc[grouped.groupby('Price Segment')['Display Score'].idxmax()]
best_port = grouped.loc[grouped.groupby('Price Segment')['Portability Score'].idxmax()]
best_total = grouped.loc[grouped.groupby('Price Segment')['Total Score'].idxmax()]

best_work['Category'] = 'Work'
best_work['Score'] = best_work['Work Score']

best_play['Category'] = 'Gaming'
best_play['Score'] = best_play['Play Score']

best_display['Category'] = 'Display'
best_display['Score'] = best_display['Display Score']

best_port['Category'] = 'Portability'
best_port['Score'] = best_port['Portability Score']

best_total['Category'] = 'General'
best_total['Score'] = best_total['Total Score']

best_brands = pd.concat([best_work, best_play, best_display,best_port, best_total]).reset_index(drop=True)
print(best_brands)

   Price Segment      Brand  Work Score  Play Score  Display Score  \
0            Low    Samsung    6.033333    3.322222       7.222222   
1      Lower Mid   GIGABYTE    7.473684    7.542105       7.105263   
2            Mid      AORUS    7.825000    8.000000       8.125000   
3      Upper Mid      AORUS    7.822222    8.200000       8.055556   
4           High  Alienware    8.221192    8.450331       7.523179   
5            Low        MSI    5.816667    4.992857       7.040476   
6      Lower Mid   GIGABYTE    7.473684    7.542105       7.105263   
7            Mid      AORUS    7.825000    8.000000       8.125000   
8      Upper Mid      AORUS    7.822222    8.200000       8.055556   
9           High      Razer    8.200000    8.515385       8.538462   
10           Low  Microsoft    4.928571    2.800000       8.428571   
11     Lower Mid  Microsoft    5.246667    2.986667       9.366667   
12           Mid  Microsoft    5.785714    3.352381       9.571429   
13     Upper Mid  Mi

In [ ]:
fig = px.bar(
    best_brands,
    x="Price Segment",
    y="Score",
    color="Category",
    text="Brand",
    title="Best Brand by Price Segment and Category",
    labels={"Score": "Average Score", "Price Segment": "Price Range", "Category": "Purpose"},
    hover_data={"Brand": True},
    color_discrete_sequence=px.colors.qualitative.Set2,
    height=400,
    barmode ='group'
)
fig.update_traces(textposition='outside')
fig.update_layout(
    font=dict(size=14),
    title_font=dict(size=18),
    legend=dict(title="Category", font_size=12),
    xaxis=dict(title="Price Segment", tickfont_size=12),
    yaxis=dict(title="Average Score", tickfont_size=12),
)
fig.show()



# 2, Collect all laptop brands available, find the highest and lowest-priced laptop for each brand.

In [ ]:
# Group by Brand and calculate highest and lowest price
brand_prices = data.groupby('Brand')['Cost'].agg(['max', 'min']).reset_index()
brand_prices.rename(columns={'max': 'Highest Price', 'min': 'Lowest Price'}, inplace=True)

        Brand  Highest Price  Lowest Price
0       AORUS        5598.00        949.00
1        ASUS        9149.00        199.99
2        Acer        7784.00        228.73
3   Alienware        6599.00       1199.00
4        Dell      982321.64        225.00
5    Dynabook        1699.95       1699.95
6    GIGABYTE        5479.00        884.83
7          HP        9186.21        177.89
8          LG        2499.00        589.99
9      Lenovo        9214.46        259.30
10        MSI        9999.00        103.64
11  Microsoft        3367.99        520.18
12  Panasonic        7809.60       2699.99
13    Porsche        2069.00       2069.00
14      Razer        6298.00       1749.99
15    Samsung        5299.00        529.00


In [96]:
# Find laptops with the highest and lowest prices for each brand
highest_priced_laptops = data.loc[data.groupby('Brand')['Cost'].idxmax()]
lowest_priced_laptops = data.loc[data.groupby('Brand')['Cost'].idxmin()]
# Merge results
brand_price_details = brand_prices.copy()
brand_price_details['Highest Priced Laptop'] = highest_priced_laptops['name'].values
brand_price_details['Lowest Priced Laptop'] = lowest_priced_laptops['name'].values
drop_dell = brand_price_details.drop(4)

In [97]:
df_melted = drop_dell.melt(id_vars=['Brand', 'Highest Priced Laptop', 'Lowest Priced Laptop'], 
                    value_vars=['Highest Price', 'Lowest Price'], 
                    var_name='Price Type', 
                    value_name='Price')
fig = px.bar(
    df_melted, 
    x="Brand", 
    y="Price", 
    color="Price Type", 
    text="Price", 
    hover_data=["Highest Priced Laptop", "Lowest Priced Laptop"], 
    labels={"Price": "Price (in USD)", "Brand": "Laptop Brand", "Price Type": "Price Type"},
    title="Highest and Lowest Prices by Laptop Brand",
    color_discrete_map={"Highest Price": "orange", "Lowest Price": "blue"},
    height=500
)
fig.show()


# 3, Evolution of Storage Type Preferences (SSD vs. HDD) Over Time

Track changes in the adoption of SSDs vs. HDDs across release years, segmented by price range. Time-series graphs and stacked area plots would reveal shifts in consumer preferences over time

# 4, Trade-off Analysis Between Performance and Battery Life in Gaming Laptops

Plot Play Score against battery life and Portability Score for gaming models. Use scatter plots or line charts to reveal the trade-off points and balance between performance and portability.



In [ ]:
data['Play Score'].mean()

0         8.3
4         8.2
8         9.6
9         9.6
12        9.6
         ... 
288578    1.8
288609    1.8
288668    1.8
288731    1.8
288997    1.6
Name: Play Score, Length: 31642, dtype: float64

In [108]:
gaming_laptops = data[data['Play Score']>data['Play Score'].mean()]
print(gaming_laptops.shape)
fig1 = px.scatter(gaming_laptops, 
                  x='Play Score', 
                  y='GPU: Power Consumption', 
                  color='Brand', 
                  title='Play Score vs Battery Life in Gaming Laptops',
                  labels={'Play Score': 'Gaming Performance (Play Score)', 
                          'Battery Life (hrs)': 'Battery Life (hours)'},
                  hover_data=['Brand'],
                  color_discrete_sequence=px.colors.qualitative.Set2)
fig2 = px.scatter(gaming_laptops, 
                  x='Play Score', 
                  y='Portability Score', 
                  color='Brand', 
                  title='Play Score vs Portability Score in Gaming Laptops',
                  labels={'Play Score': 'Gaming Performance (Play Score)', 
                          'Portability Score': 'Portability Score'},
                  hover_data=['Brand'],
                  color_discrete_sequence=px.colors.qualitative.Set2)
fig1.show()
fig2.show()

(15774, 64)


# 5, Laptops with recently released CPUs and GPUs are more likely to offer higher performance scores but at a higher price, while older component configurations may offer the best price-performance balance for budget-conscious consumers.

Plot Total Score and Cost for laptops with different CPU/GPU release timelines to identify the ideal price-performance ratios. Use scatter plots with regression lines to highlight the price-performance trade-off for each configuration.

# 6, Price over the specific-task laptops(laptops that have specific score in the top 5-10% largest)

In [ ]:
specific_score_column = 'Display Score'
top_5_percent_threshold = data[specific_score_column].quantile(0.95)
top_10_percent_threshold = data[specific_score_column].quantile(0.90)
top_5_percent_laptops = data[data[specific_score_column] >= top_5_percent_threshold]
top_10_percent_laptops = data[data[specific_score_column] >= top_10_percent_threshold]
fig_5 = px.histogram(top_5_percent_laptops, 
                      x='Cost', 
                      title=f'Cost Distribution of Laptops with Top 5% {specific_score_column}',
                      labels={'Cost': 'Cost (USD)', f'{specific_score_column}': f'Top 5% {specific_score_column}'},
                      color='Brand',
                      color_discrete_sequence=px.colors.qualitative.Set2)
fig_10 = px.histogram(top_10_percent_laptops, 
                       x='Cost', 
                       title=f'Cost Distribution of Laptops with Top 10% {specific_score_column}',
                       labels={'Cost': 'Cost (USD)', f'{specific_score_column}': f'Top 10% {specific_score_column}'},
                       color='Brand',
                       color_discrete_sequence=px.colors.qualitative.Set2)
fig_5.show()
fig_10.show()

In [118]:
score_columns = ['Play Score', 'Work Score', 'Portability Score','Display Score','Total Score']
top_5_mean_prices = {}
for score_column in score_columns:
    top_5_percent_threshold = data[score_column].quantile(0.95)
    top_5_laptops = data[data[score_column] >= top_5_percent_threshold]
    mean_price = top_5_laptops['Cost'].mean()
    top_5_mean_prices[score_column] = mean_price
top_5_prices_df = pd.DataFrame(list(top_5_mean_prices.items()), columns=['Score Type', 'Mean Price'])
fig = px.bar(top_5_prices_df,
             x='Score Type', 
             y='Mean Price',
             title="Mean Price of Laptops in Top 5% for Each Score",
             labels={'Mean Price': 'Mean Price (USD)', 'Score Type': 'Score Type'},
             color='Score Type',
             color_discrete_sequence=px.colors.qualitative.Set2,
             height=400)
fig.show()

# 7, The difference in the architecture of laptops for each specific-task 

can be done in prediction files

# 8, Any trends in use of GPU and CPU for each task? For example, have a trend recently likely to use the type of archtitecture of CPU for a brand for a task of gaming? or something else in architecture that correlated to CPU and GPU( related to date)???